爬蟲 需要的套件

In [46]:
!pip install bs4
!pip install requests
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import json
import time
import re
from urllib import request
import random

Redis 需要的套件

In [2]:
!pip install redis
import redis
from hashlib import md5

In [34]:
class SimpleHash(object):
    def __init__(self, cap, seed):
        self.cap = cap
        self.seed = seed

    def hash(self, value):
        ret = 0
        for i in range(len(value)):
            ret += self.seed * ret + ord(value[i])
        return (self.cap - 1) & ret


class BloomFilter(object):
    def __init__(self, host='redis', port=6379, db=0, blockNum=1, key='bloomfilter'):
        """
        :param host: the host of Redis
        :param port: the port of Redis
        :param db: witch db in Redis
        :param blockNum: one blockNum for about 90,000,000; if you have more strings for filtering, increase it.
        :param key: the key's name in Redis
        """
        self.server = redis.Redis(host=host, port=port, db=db)
        self.bit_size = 1 << 31  # Redis的String类型最大容量为512M，现使用256M
        self.seeds = [5, 7, 11, 13, 31, 37, 61]
        self.key = key
        self.blockNum = blockNum
        self.hashfunc = []
        for seed in self.seeds:
            self.hashfunc.append(SimpleHash(self.bit_size, seed))

    def isContains(self, str_input):
        if not str_input:
            return False
        m5 = md5()
        m5.update(str_input)
        str_input = m5.hexdigest()
        ret = True
        name = self.key + str(int(str_input[0:2], 16) % self.blockNum)
        for f in self.hashfunc:
            loc = f.hash(str_input)
            ret = ret & self.server.getbit(name, loc)
        return ret

    def insert(self, str_input):
        m5 = md5()
        m5.update(str_input)
        str_input = m5.hexdigest()
        name = self.key + str(int(str_input[0:2], 16) % self.blockNum)
        for f in self.hashfunc:
            loc = f.hash(str_input)
            self.server.setbit(name, loc, 1)

BloomFilter方法:處理資料去重複

In [44]:
bf = BloomFilter()
# url = "https://tw.news.appledaily.com/politics/realtime/20181125/1473059/"
# url = url.encode('utf-8')
# if bf.isContains(url):   # 判断字符串是否存在
#     print('url is exists!')
# else:
#     print('url is not exists!')
#     bf.insert(url)

In [5]:
#BloomFilter的key
bf.key

'bloomfilter'

In [5]:
pool = redis.ConnectionPool(host='redis', port=6379, db=0)
r = redis.StrictRedis(connection_pool=pool)

In [6]:
#搜尋redis裡面有哪些keys
keys = r.keys()
print(keys)

[b'bloomfilter0']


In [27]:
#刪除redis裡key為XXX
r.delete("bloomfilter0")

1

傳送log 需要的套件

In [48]:
!pip install python-logstash
import logging
import logstash
import sys

In [29]:
host = "logstash"
crawler_logger = logging.getLogger('crawler_logger')
crawler_logger.setLevel(logging.INFO)

# TCP
crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))

Kafka Producer 需要的套件

In [10]:
!pip install kafka
from kafka import KafkaProducer
import sys
from kafka.errors import KafkaError

連線Kafka

In [11]:
producer = KafkaProducer(
    # Kafka集群在那裡?
    bootstrap_servers = ["kafka1:29092"],
    # 指定msgKey的序列化器, 若Key為None, 無法序列化, 透過producer直接給值
    key_serializer = str.encode,
    # 指定msgValue的序列化器
    value_serializer = lambda m: json.dumps(m).encode('utf-8')
)

TVBS 爬蟲方法

In [16]:
def tvbs_crawler2kafka():
    ###TVBS爬蟲開始###
    url = "https://news.tvbs.com.tw/politics/"
    response = urlopen(url)
    soup = BeautifulSoup(response)
    news_ul = soup.find_all("ul", id = "block_pc")[0]
    news_a = news_ul.find_all("a")

    news_no_string_list = []
    for a in news_a:
        news_no_string = a["data-news-id"]
        news_no_string_list.append(news_no_string)

    news_no_list = []
    for n in news_no_string_list:
        news_no_int = n.strip('\'')
        news_no_list.append(news_no_int)

    #json_data = []
    for news_no in news_no_list:

        print("新聞來源:", "TVBS新聞台")

        news_url = "https://news.tvbs.com.tw/politics/" + news_no
        print("新聞網址:", news_url)

        
        #判斷網址是否爬過
        bf = BloomFilter()
        # Unicode-objects must be encoded before hashing
        news_url_u = news_url.encode('utf-8')
        if bf.isContains(news_url_u):   # 判断字符串是否存在
            print('url is exists!')
            continue
        else:
            print('url is not exists!')
            bf.insert(news_url_u)
        

        req = requests.get(news_url)
        # requests如果找不到指定編碼，會猜測網頁編碼，有時會形成亂碼，故給指定編碼utf-8
        req.encoding = ("utf-8")
        soup = BeautifulSoup (req.text, 'html.parser')

        title = soup.find("h1", class_ = "margin_b20").text
        title = re.sub(r"\u3000", " ", title)
        title = re.sub(r"\xa0", " ", title)
        print("新聞標題:", title)

        date = soup.find("div", class_ = "icon_time time leftBox2").text
        print("發佈時間:", date)

        author = []
        author_text = soup.find("h4", class_ = "font_color5 leftBox1").text
        author_text = re.sub(r"攝影.*報導", "", author_text)
        author.append(author_text)
        print("作者:", author)

        content = soup.find("div", class_ = "h7 margin_b20").text
        content = content.replace("\t", "").replace("   ","").replace("\n", "").replace("（中央社）","")
        content = content.replace("最HOT話題在這！想跟上時事，快點我加入TVBS新聞LINE好友！ ","")
        content = content.replace("最HOT話題在這！想跟上時事，","").replace("快點我加入TVBS新聞LINE好友！ ","")
        content = content.replace("TVBS最新大數據分析","").replace("看完整內容快點我加入TVBS新聞LINE好友！","")
        print("內文:", content)

        keyword = soup.find("div", class_ = "adWords").text
        keyword = keyword.replace("\t", "")
        keyword = keyword.replace(",","")
        keyword = keyword.replace("\n", "").replace("編輯  ","").replace(" 報導","")
        keyword = keyword.replace("記者","").replace("      ",",")
        keyword = keyword.split(",")
        print("關鍵字", keyword)

        img = soup.find("div", class_ = "margin_b20")
        img_url = img.find('img')['src']
        print("圖片網址:", img_url)

        print("\n")

        j = {"source": "TVBS新聞台" ,"url": news_url, "title": title, "date_": date, "author": author, "content": content, 
             "kw": keyword, "img_url": img_url}
        time.sleep(1)
        
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲成功傳送至Elasticsearch的log訊息
        crawler_logger.info('python-crawler-logstash:TVBS crawler success!!')
        crawler_logger.handlers.clear()
        #json_data.append(j)
    ###TVBS爬蟲結束###
    

   
        # 步驟2.指定想要發佈訊息的topic名稱
        topic_name = "test"

        try:
            print("Start sending messages ...")
            # 步驟3.產生要發佈到Kafka的訊息
            # - 參數  # 1: topicName
            # - 參數  # 2: msgKey
            # - 參數  # 3: msgValue

            producer.send(topic = topic_name, key = "tvbs", value = j)
            print("Message sending completed!")
        except Exception as e:
            # 錯誤處理
            e_type, e_value, e_traceback = sys.exc_info()
            print("type ==> %s" % (e_type))
            print("value ==> %s" % (e_value))
            print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
            print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
            print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
            # 爬蟲失敗傳送至Elasticsearch的log訊息
            crawler_logger.error('python-crawler-logstash :TVBS crawler error message!!')
            crawler_logger.handlers.clear()

SETN爬蟲方法

In [17]:
def setn_crawler2kafka():
    ###SETN爬蟲開始###
    page = 1
    #json_data = []
    while True:
        page_url = "https://www.setn.com/ViewAll.aspx?PageGroupID=6&p=" + str (page)
        print ("Processing:", page_url)

        response = urlopen (page_url)
        bs = BeautifulSoup (response)
        titles = bs.find_all ("h3", "view-li-title")

        if len (titles) == 0:
            break

        for t in titles:

            print("新聞來源:", "SETN三立新聞網")

            news_url = "https://www.setn.com" + t.find ("a")["href"]
            print("新聞網址:", news_url)
            
            
            #判斷網址是否爬過
            bf = BloomFilter()
            # Unicode-objects must be encoded before hashing
            news_url_u = news_url.encode('utf-8')
            if bf.isContains(news_url_u):   # 判断字符串是否存在
                print('url is exists!')
                continue
            else:
                print('url is not exists!')
                bf.insert(news_url_u)
                

            req = requests.get (news_url)
            bs = BeautifulSoup (req.text, 'html.parser')

            title = bs.find('h1', class_ = "news-title-3").text
            title = re.sub(r"\u3000", " ", title)
            print("新聞標題:", title)

            date = bs.find("time", class_ = "page-date").text.strip()
            print("發佈時間:", date)

            author = []
            a_divs = bs.find("div", id="Content1")
            first_p = a_divs.find("p")
            author_text = first_p.text
            #找第一段有沒有含"／"，如果沒有給予空字串
            if author_text.find("／") == -1:
                author_text = ""
            #如果作者是空字串，給予空list
            if author_text == "":
                author = []
            else:
                author.append(author_text)
            print("作者:", author)

            c_divs = bs.find('div', itemprop='articleBody')
            content_all = c_divs.find_all ("p")
            content = ""
            for c in content_all:
                if c.attrs == {}:
                    content = content + c.text
                    #print(c.text)

                    c_divs = bs.find('div', itemprop='articleBody')
            content_all = c_divs.find_all ("p")
            content = ""
            for c in content_all:
                if c.attrs == {}:
                    content = content + c.text
                    #print(c.text)
            print("內文:", content)

            keyword = []
            k_divs = bs.find("div", class_="keyword page-keyword-area")
            k_strong = k_divs.find_all("strong")
            for k in k_strong:
                keyword_text = k.text
                keyword.append(keyword_text)
            print("關鍵字:", keyword)

            img_p = bs.find("p", style = "text-align: center;")
            img_url = img_p.find("img")["src"]
            print("圖片網址:", img_url)

            print("\n")

            j = {"source": "SETN三立新聞網" ,"url": news_url, "title": title, "date_": date, "author": author, "content": content, 
                 "kw": keyword, "img_url": img_url}
            time.sleep (1)
            
            # TCP連線到logstash
            crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
            # 爬蟲成功傳送至Elasticsearch的log訊息
            crawler_logger.info('python-crawler-logstash:SETN crawler success!!')
            crawler_logger.handlers.clear()
            #json_data.append(j)
    ###SETN爬蟲結束###
    
            # 步驟2.指定想要發佈訊息的topic名稱
            topic_name = "test"

            try:
                print("Start sending messages ...")
                # 步驟3.產生要發佈到Kafka的訊息
                # - 參數  # 1: topicName
                # - 參數  # 2: msgKey
                # - 參數  # 3: msgValue

                producer.send(topic = topic_name, key = "setn", value = j)
                print("Message sending completed!")
            except Exception as e:
                # 錯誤處理
                e_type, e_value, e_traceback = sys.exc_info()
                print("type ==> %s" % (e_type))
                print("value ==> %s" % (e_value))
                print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
                print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
                print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
                # 爬蟲失敗傳送至Elasticsearch的log訊息
                crawler_logger.error('python-crawler-logstash :SETN crawler error message!!')
                crawler_logger.handlers.clear()
        time.sleep (2)
        page = page + 1

AppleDaily爬蟲方法

In [18]:
def appledaily_crawler2kafka():
    ###AppleDaily爬蟲開始###
    for page in range(1, 12):
        time.sleep(2)
        href = "https://tw.news.appledaily.com/politics/realtime/" + str(page)
        res = requests.get(href)
        html = BeautifulSoup(res.text)

        all_news_1 =  html.find_all("ul", class_= "rtddd slvl")
        for all_news in all_news_1:
            news = all_news.find_all("a")
            print("第", page, "頁")
            for n in news:

                print("新聞來源:", "蘋果日報")

                #my_news = {}
                news_url ="https://tw.news.appledaily.com/" + str(n["href"])
                print("新聞網址:", news_url)
                
                #判斷網址是否爬過
                bf = BloomFilter()
                # Unicode-objects must be encoded before hashing
                news_url_u = news_url.encode('utf-8')
                if bf.isContains(news_url_u):   # 判断字符串是否存在
                    print('url is exists!')
                    continue
                else:
                    print('url is not exists!')
                    bf.insert(news_url_u)

                news_per = requests.get(news_url)
                bs = BeautifulSoup(news_per.text)

                title = bs.find("h1").text
                title = re.sub(r"\u3000", " ", title)
                print("新聞標題:", title)           

                date = bs.find("div", class_="ndArticle_creat").text.replace("出版時間：", "")
                print("發佈時間:", date)

                if not bs.find("div", class_="ndgKeyword") == None:
                    key_word = bs.find("div", class_="ndgKeyword").find_all("a")
                    key_list = []
                    for k in key_word:
                        key_list.append(k.text)
                else:
                    key_list = []       

                content_dir = bs.find("div", class_="ndArticle_margin")
                for a in content_dir.find_all("a"):
                    if not a == None:
                        a.extract()
                for s in content_dir.find_all("span"):
                    if not s == None:
                        s.extract()
                for i in content_dir.find_all("iframe"):
                    if not i == None:
                        i.extract()
                for d in content_dir.find_all("div"):
                    if not d == None:
                        d.extract()
                for st in content_dir.find_all("style"):
                    if not st == None:
                        st.extract()
                content = content_dir.text.strip()
                content = content.replace("）", ")").replace("（", "(").replace("／", "/").replace("╱", "/")       
                pat = re.compile(r"\(((.{0,15})/.{0,8}[\u5831][\u5c0e])\)")
                pat_2 = re.compile(r"【((.{0,15})/.{0,8}[\u5831][\u5c0e])】")
                ans_news = pat.search(content)
                ans_news_2 = pat_2.search(content)
                if ans_news == None:
                    author = []
                elif not ans_news_2 == None:
                    author = ans_news[2].split("、")
                else:
                    #print(ans_news[2])
                    author = ans_news[2].split("、")
                    content = content.split(ans_news[0])[0]
                print("作者:", author)   
                print("內文:", content)  
                print("關鍵字:", key_list)

                img_box = bs.find("div", class_="ndAritcle_headPic")
                if not img_box == None:
                    img_url = img_box.find("img")['src']
                else:
                    img_url = ""
                print("圖片網址:", img_url)

                print("\n")

                j = {"source": "蘋果日報" ,"url": news_url, "title": title, "date_": date, "author": author, 
                     "content": content, "kw": key_list, "img_url": img_url}
                time.sleep(random.randint(1, 2))
                
                # TCP連線到logstash
                crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                # 爬蟲成功傳送至Elasticsearch的log訊息
                crawler_logger.info('python-crawler-logstash:AppleDaily crawler success!!')
                crawler_logger.handlers.clear()
                #json_data.append(j)
    ###AppleDaily爬蟲結束###

                # 步驟2.指定想要發佈訊息的topic名稱
                topic_name = "test"

                try:
                    print("Start sending messages ...")
                    # 步驟3.產生要發佈到Kafka的訊息
                    # - 參數  # 1: topicName
                    # - 參數  # 2: msgKey
                    # - 參數  # 3: msgValue

                    producer.send(topic = topic_name, key = "appledaily", value = j)
                    print("Message sending completed!")
                except Exception as e:
                    # 錯誤處理
                    e_type, e_value, e_traceback = sys.exc_info()
                    print("type ==> %s" % (e_type))
                    print("value ==> %s" % (e_value))
                    print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
                    print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
                    print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
                    # 爬蟲失敗傳送至Elasticsearch的log訊息
                    crawler_logger.error('python-crawler-logstash :AppleDaily crawler error message!!')
                    crawler_logger.handlers.clear()

NOWnews爬蟲方法

In [24]:
def nownews_crawler2kafka():
    ###NOWnews爬蟲開始###
    for page in range(1,6):
        href = 'https://www.nownews.com/cat/politics/page/'+str(page)

        response = requests.get(href)
        soup=BeautifulSoup(response.text)
        all_news = soup.find_all("div",class_="td_block_inner tdb-block-inner td-fix-index")

        news = all_news[1].find_all("h3",class_="entry-title")
        for n in news:
            try:
                print("新聞來源:", "NOWnews")

                news_url = n.find("a")["href"]
                print("新聞網址:", news_url)
                
                
                #判斷網址是否爬過
                bf = BloomFilter()
                # Unicode-objects must be encoded before hashing
                news_url_u = news_url.encode('utf-8')
                if bf.isContains(news_url_u):   # 判断字符串是否存在
                    print('url is exists!')
                    continue
                else:
                    print('url is not exists!')
                    bf.insert(news_url_u)
                    

                response = requests.get(news_url)
                html = BeautifulSoup(response.text)
                title = html.find("h1",class_="entry-title").text
                title = re.sub("\u3000", " ", title)
                print("新聞標題:", title)

                date = html.find("time",class_ = "entry-date").text
                print("發佈時間:", date)

                author = []
                author_text = html.find("div",class_="td-post-author-name").text
                author_text = author_text.split("／")[0]
                author_text = author_text.split("/")[0].replace("記者","").replace("\n","").replace(" ","")
                author.append(author_text)
                print("作者:", author)

                te = html.find("div", class_="td-post-content").contents
                #print(te)
                #print(te[0].name == None)
                content = ""
                for a in te:
                    if a.name == "p":
                        content = content + a.text
                print("內文:", content)

                #處理keyword
                if not html.find("ul",class_="td-tags") == None:
                    kw = html.find("ul",class_="td-tags").find_all("a")
                    keyword = []
                    for k in kw:
                        keyword.append(k.text)
                else:
                    keyword = []
                print("關鍵字:", keyword)

                img_url = html.find("div",class_="td-post-featured-image")
                img_url = img_url.find("img")["src"]
                print("圖片網址:", img_url)

            except:
                print("錯誤:", title, news_url)

            print("\n")

            j = {"source": "NOWnews" ,"url": news_url, "title": title, "date_": date, "author": author, "content": content, 
                 "kw": keyword, "img_url": img_url}
            time.sleep(random.randint(0,2))
            
            # TCP連線到logstash
            crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
            # 爬蟲成功傳送至Elasticsearch的log訊息
            crawler_logger.info('python-crawler-logstash:NOWnews crawler success!!')
            crawler_logger.handlers.clear()
            #json_data.append(j)
    ###NOWnews爬蟲結束###
    
            # 步驟2.指定想要發佈訊息的topic名稱
            topic_name = "test"

            try:
                print("Start sending messages ...")
                # 步驟3.產生要發佈到Kafka的訊息
                # - 參數  # 1: topicName
                # - 參數  # 2: msgKey
                # - 參數  # 3: msgValue

                producer.send(topic = topic_name, key = "nownews", value = j)
                print("Message sending completed!")
            except Exception as e:
                # 錯誤處理
                e_type, e_value, e_traceback = sys.exc_info()
                print("type ==> %s" % (e_type))
                print("value ==> %s" % (e_value))
                print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
                print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
                print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
                # 爬蟲失敗傳送至Elasticsearch的log訊息
                crawler_logger.error('python-crawler-logstash :NOWnews crawler error message!!')
                crawler_logger.handlers.clear()

STORM爬蟲方法

In [32]:
def storm_crawler2kafka():
    ###STORM爬蟲開始###
    json_data = []    
    for page in range(1,5):
        href = 'https://www.storm.mg/category/118/'+str(page)

        response = urlopen(href)
        html = BeautifulSoup(response)
        #找到特定範圍在選取網址列
        newf = html.find("div",class_ = "middle_category_cards")
        #選取所有單篇網址列
        news = newf.find_all("div",class_ = "category_card")
        #迴圈抓取整頁新聞連結
        for r in news:

            print("新聞來源:", "風傳媒 THE STORM MEDIA")

            news_url = r.find("a",class_ = "card_link")["href"]
            print("新聞網址:", news_url)
            
            
            #判斷網址是否爬過
            bf = BloomFilter()
            # Unicode-objects must be encoded before hashing
            news_url_u = news_url.encode('utf-8')
            if bf.isContains(news_url_u):   # 判断字符串是否存在
                print('url is exists!')
                continue
            else:
                print('url is not exists!')
                bf.insert(news_url_u)
                

            response = urlopen(news_url)
            html = BeautifulSoup(response)

            title = html.find("h1", id = "article_title").text
            print("新聞標題:", title)

            date = html.find("span", class_="info_time").text
            print("發佈時間:", date)

            author = []
            author_text = html.find("span", class_="info_author").text.replace("新新聞","")
            author.append(author_text)
            print("作者:", author)

            content = html.find("div", id="CMS_wrapper").text.replace("\n","").replace("\t","")
            print("內文:", content)

            keyword = html.find("div", id="tags_list_wrapepr").text.split()
            print("關鍵字:", keyword)

            img_url = html.find("img", id="feature_img")["src"]
            print("圖片網址:", img_url)

            print("\n")

            time.sleep (1)

            j = {"source": "風傳媒 THE STORM MEDIA" ,"url": news_url, "title": title, "date_": date, "author": author, 
                 "content": content, "kw": keyword, "img_url": img_url}
            time.sleep(random.randint(0,2))
            
            # TCP連線到logstash
            crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
            # 爬蟲成功傳送至Elasticsearch的log訊息
            crawler_logger.info('python-crawler-logstash:STORM crawler success!!')
            crawler_logger.handlers.clear()
            #json_data.append(j)
    ###STORM爬蟲結束###
    
            # 步驟2.指定想要發佈訊息的topic名稱
            topic_name = "test"

            try:
                print("Start sending messages ...")
                # 步驟3.產生要發佈到Kafka的訊息
                # - 參數  # 1: topicName
                # - 參數  # 2: msgKey
                # - 參數  # 3: msgValue

                producer.send(topic = topic_name, key = "storm", value = j)
                print("Message sending completed!")
            except Exception as e:
                # 錯誤處理
                e_type, e_value, e_traceback = sys.exc_info()
                print("type ==> %s" % (e_type))
                print("value ==> %s" % (e_value))
                print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
                print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
                print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
                # 爬蟲失敗傳送至Elasticsearch的log訊息
                crawler_logger.error('python-crawler-logstash :STORM crawler error message!!')
                crawler_logger.handlers.clear()

ETtoday爬蟲方法

In [41]:
def ettoday_crawler2kafka():
    ###ETtoday爬蟲開始###
    #json_data = []
    for page in range(1, 1007):
        url = "https://www.ettoday.net/news_search/doSearch.php?keywords=%E6%94%BF%E6%B2%BB&kind=1" + "&idx=1&page=" + str(
            page)
        print("第", page, "頁"," ","Processing: ", url)
        try:
            response = urlopen(url)
        except HTTPError:
            print("大概是結束了")
            break
        html = BeautifulSoup(response)

        all_ar = html.find("div", class_="result_archive")

        val = all_ar.find_all("div", class_="archive clearfix")

        for u in val:

            print("新聞來源:", "ETtoday")

            box2 = u.find("div", class_="box_2")
            news_url = box2.find("a")["href"]
            print("新聞網址:", news_url)
            
            
            #判斷網址是否爬過
            bf = BloomFilter()
            # Unicode-objects must be encoded before hashing
            news_url_u = news_url.encode('utf-8')
            if bf.isContains(news_url_u):   # 判断字符串是否存在
                print('url is exists!')
                continue
            else:
                print('url is not exists!')
                bf.insert(news_url_u)
                

            a_response = urlopen(news_url)
            a_html = BeautifulSoup(a_response, "html.parser")

            title = a_html.find("h1", class_="title").text
            title = re.sub("\u3000", " ", title)
            print("新聞標題:", title)

            date = a_html.find("time", class_="date").text.strip()
            date = re.sub("年", "/", date)
            date = re.sub("月", "/", date)
            date = re.sub("日", "", date)
            print("發佈時間:", date)


            a_story = a_html.find("div", class_="story")                
            content_p = a_story.find_all("p")
            author_c = ""
            for c_p in content_p:
                #如果c_p裡面的第一個格子是None
                if c_p.contents[0].name == None:
                    author_c = author_c + c_p.text
                elif c_p.contents[0].name == "span":
                    author_c = author_c + c_p.text
            author = []
            if re.search(r".*／.?.?報導", author_c) == None:
                author = []
            else:
                #因為作者與內文會放在同一個格子，所以將內文中會抓到的作者部分在內文處理時刪除
                content_del = re.search(r".*／.?.?報導", author_c).group()
                author_text = re.search(r".*／.?.?報導", author_c).group() 
            author.append(author_text)
            print("作者:", author)              

            content = ""
            for c in content_p:
                if c.contents[0].name == None:
                    content = content + c.text
                    content = content.replace("\u3000", " ")      
                    content = content.replace(content_del, "")            
            print("內文:", content)


            keyword = []
            a_web = a_html.find("article")
            a_kw = a_web.find("p", class_="tag")
            kw = a_kw.find_all("a", target="_blank")
            for k in kw:
                kw_n = k.text
                keyword.append(kw_n)
            if keyword == [""]:
                keyword = []
            print("關鍵字:", keyword)


            if not a_story.find("img")['src'] == None:
                img_url = a_story.find("img")['src']
                img_url = "https:" + img_url
            else:
                img_url = None
            print("圖片網址:", img_url)

            print("\n")

            j = {"source": "ETtoday" ,"url": news_url, "title": title, "date_": date, "author": author, "content": content, 
                 "kw": keyword, "img_url": img_url}
            time.sleep(1)
            
            # TCP連線到logstash
            crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
            # 爬蟲成功傳送至Elasticsearch的log訊息
            crawler_logger.info('python-crawler-logstash:ETtoday crawler success!!')
            crawler_logger.handlers.clear()
            #json_data.append(j)
    ###ETtoday爬蟲結束###
    
            # 步驟2.指定想要發佈訊息的topic名稱
            topic_name = "test"

            try:
                print("Start sending messages ...")
                # 步驟3.產生要發佈到Kafka的訊息
                # - 參數  # 1: topicName
                # - 參數  # 2: msgKey
                # - 參數  # 3: msgValue

                producer.send(topic = topic_name, key = "ettoday", value = j)
                print("Message sending completed!")
            except Exception as e:
                # 錯誤處理
                e_type, e_value, e_traceback = sys.exc_info()
                print("type ==> %s" % (e_type))
                print("value ==> %s" % (e_value))
                print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
                print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
                print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
                # 爬蟲失敗傳送至Elasticsearch的log訊息
                crawler_logger.error('python-crawler-logstash :ETtoday crawler error message!!')
                crawler_logger.handlers.clear()

主程式

In [28]:
#!pip install schedule
import schedule
import time
import datetime
import threading


#def job():
#    print("I'm working...")
#schedule.every(1).minutes.do(job)

#schedule.every().day.at("16:42").do(tvbs_crawler2kafka)
#schedule.every().day.at("17:51").do(setn_crawler2kafka)
#schedule.every().day.at("22:27").do(appledaily_crawler2kafka) 
#schedule.every().day.at("22:27").do(nownews_crawler2kafka)
#schedule.every().day.at("22:27").do(storm_crawler2kafka())


while True:
    schedule.run_pending()
    time.sleep(1)


第 1 頁
新聞來源: 蘋果日報
新聞網址: https://tw.news.appledaily.com//politics/realtime/20181221/1487411/
新聞標題: 韓國瑜賽馬場創意惹議 觀光局：希望明年1月赴港考察
發佈時間: 2018/12/21 21:17
作者: ['周昭平']
內文: (新增：準觀光局長潘恆旭說明)準高雄市長韓國瑜昨與中華民國工商建研會座談時，提到未來想把已停工的中油五輕廠，引進賽馬、發展相關產業鏈。此驚人發想立即引發爭議。準高雄市府新聞局長王淺秋今上午出面說明強調，昨天韓國瑜提賽馬只是創意發想，場地也不一定會在中油廠區，需進一步透過專業評估與合乎中央法規，才可能成為確定執行計畫。王淺秋說，昨韓國瑜會在會中提及賽馬，主要是因高雄目前財務現實狀況，要想辦法創造稅收、在價值供應鏈上創造最多效益，賽馬場只是評估考量之一，不過因實際上世界多國都有賽馬場，創造經濟供應鏈驚人，且有機會做公益，所以才會成為考量發展產業之一，雖說目前為止都只是創意發想階段，但觀光局已表示，希望明年一月可以安排到香港考察，同時針對法規問題了解，若評估可行就會來推動。此外針對韓國瑜昨在會中提到有台灣人曾任香港馬會會長，遭網友質疑經查香港馬會根本沒有台灣人，酸韓國瑜遭詐騙。對此，準高市府觀光局局長潘恆旭說，韓所提台灣人會長應是前香港馬會行政總裁黃至剛，他曾到台灣成大念書，後獲美國博士學位後回到台灣工作多年，1996年獲聘香港馬會行政總裁，2007年退休。但黃至剛是否為台灣人，這部分他並不清楚。潘恆旭說，黃至剛與韓國瑜之前有透過相熟的朋友聊到賽馬的經驗，但至於是否實際碰面他並不知情，此外黃至剛的職稱應是行政總監，韓國瑜講會長是口誤。潘恆旭說，因團隊還未就任，韓國瑜對此僅在構想階段，曾提過，但內部也還未有縝密的沙盤推演與輿情的配套，拋出想法是想要振興高雄經濟，畢竟賽馬可創造兩萬工作機會與很高產值，強調其原始概念是如此，不是說非推賽馬不可。至於明年一月去香港實地考察是否成行，會等到就任後，團隊評估覺得可試試看，確實可當成一個可評估的案子，就會成行。另外界質疑，中油五輕廠區多為汙染整治控制場址，該處若興建馬場時光是整治就要花17年。王淺秋回應，賽馬場並「不一定要在高雄煉油廠，很多其他場地可以考量！」強調引進不只是為賽馬這件事，而是運動型態供應鏈可以帶起的產業

Start sending messages ...
Message sending completed!
新聞來源: 蘋果日報
新聞網址: https://tw.news.appledaily.com//politics/realtime/20181221/1486688/
新聞標題: 慶祝韓國瑜就職 韓粉今起美食接力看這裡
發佈時間: 2018/12/21 18:52
作者: ['吳慧芬', '王勇超']
內文: (新增：過音影片)韓國瑜將於下周二上任，從本周五起到25日就職日，天天都有熱心的「韓粉」免費招待美食，本周五有2千顆肉粽、2千碗廣東煲湯，周六有千杯杏仁茶，23日有5千碗滷肉飯、5千顆客家野薑花粽與888瓶冷泡茶，24日有1千份雞排、700份魚排，25日還有200杯綠豆冰沙可以喝，支持韓國瑜共拼經濟。選戰時大力支持韓國瑜的婦女界人士潘金英，今天下午在左營區自由路黃昏市場，準備了2千份糯米雞、2千分廣東煲湯，要免費發送給民眾，消息傳出吸引許多民眾前來排隊，下午兩點不到，就排了200多人，其中排第一號的劉先生(45歲)在胸口掛著自己用鐵絲製作的韓國瑜吊飾，他表示，他今天早上九點就來排隊，要慶祝韓市長當選「貨出得去，人進得來，高雄發大財！」英文老師謝小姐表示，她是韓國瑜的支持者，之前很多次「貪食蛇」(指韓粉慶祝韓國瑜當選，發放美食的排隊隊伍)都因為沒有時間排隊而沒參加，今天剛好有時間，就來參加，「這種活動帶動高雄的經濟！」在現場發送咖啡的業者表示，今天很高興，藉由慶祝韓國瑜當選市長的機會，特別現場請大家喝咖啡，「就像韓市長說的，東西要品質好，我們就推銷出去！」蔡姓民眾喝了咖啡，直呼：「香醇好喝！」韓國瑜選前喊出拼經濟口號，當選後「韓總美食」均受惠，韓國瑜光顧過的小吃店幾乎業績都提升，還有韓粉特製「韓總美食版清單」，讓外縣市遊客，得以按圖索驥從苓雅區滷肉飯，吃到旗山冰店，遍嚐高雄美食，帶動小吃經濟。 另有大方民眾，慶祝韓國瑜下周二上任，從本周五起天天招待美食。這包括一名住在台北韓粉郭先生，本要在25日韓國瑜當選日，於愛河畔發放1萬碗滷肉飯，受限於場地，改在23日中午於國民黨高市黨部前發送5000碗魯肉飯、5000顆客家野薑花粽，另高雄在地財記居冷泡茶，當日也將一併贈送888瓶茶飲。
關鍵字: ['韓國瑜', '韓總美食', '就職典禮', '美食接力']
圖片

KeyboardInterrupt: 

In [45]:
ettoday_crawler2kafka()

第 1 頁   Processing:  https://www.ettoday.net/news_search/doSearch.php?keywords=%E6%94%BF%E6%B2%BB&kind=1&idx=1&page=1
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181224/1339321.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181224/1338839.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181224/1339228.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181224/1339294.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181224/1339271.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181224/1339214.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181224/1339177.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181224/1339057.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181224/1339086.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181224/1339035.htm
url is exists!
第 2 頁   Processing:  https://www

新聞標題: 影／不能說假新聞男大生沒有關係！謝長廷深夜PO文回應了：為幫蘇啓誠解套
發佈時間: 2018/12/23 22:58
作者: ['政治中心／綜合報導']
內文: 前大阪辦事處處長蘇啓誠遺孀日前公開聲明，強調丈夫是在完成上級交代之檢討報書後，表明「不想受到羞辱」之遺言，以死明志。對此，駐日代表謝長廷23日在臉書表示，看了蘇啓誠「願坦然受處」的檢討報告後，便「鍥而不捨」的查出假消息的散波者，目的就是要幫他解套，而非外界所指的卸責。謝長廷指出，他在9月19日看到9月10日蘇啓誠寫的「願坦然受處」的檢討報告，內容是以巴士是載到泉佐野站及guruguru的電話為真的前提而檢討，他認為，若此事為真，那麼蘇啓誠願受處分的檢討報告會變成可以處分的根據，問題只是處分輕重及是否過當的問題。謝長廷表示，關西機場事件受困旅客輸運完畢的第一天（9/6），他就在臉書指出所謂「中國領事館派車優先救出中國人」是假新聞，後來對照網路資料，更發現guruguru講說他打電話給大阪辦事處遭到冷漠嘲諷的事情，也應該是假消息。謝長廷說，自己鍥而不捨追出假新聞率先散播者guruguru的身分，以及他打電話是假的，還有坐到泉佐野車站也是假的真相，「正是為蘇處長自我檢討報告的前提解套」，因為，蘇啓誠在檢討報告中說，自己沒有前往車站關心難辭其咎，謝長廷認為，外界追究檢討報告是否被迫？上級是否受新聞誤導而處分？處分是否過當？這些問題，既然雙方都通聯記錄可查，應該很簡單可以查明。謝長廷強調，如查出蘇啓誠輕生之前有接獲上級壓力電話而不甘受辱輕生，那應該是直接促使他尋短最後一根稻草，而不能說假新聞沒有關係，現在guruguru尋求國民黨立委保護，在政黨刻意庇護下，當時利用假新聞率先攻擊阪辦事處的委員們一下子説要為蘇啓誠申寃，但一下子又要保護假新聞的造謠者，竟然說guruguru講的也都是事實。謝長廷質疑，如果這樣，那為什麼不勸他堂堂正正出來解釋清楚，他為什麼一再逃避說謊，扯什麼帳號被盜用呢？「一切都是事實」這種說法是要陷蘇前處長於不義？還是真的要為他申寃？令人費解。針對連日來外界對蘇啓誠輕生的各種揣測，謝長廷表示，媒體引用許多「知情人士、關鍵人士、消息靈通人士、可靠人士的話，講了不少猜測，情節也許引人，但多數也是不可靠的假消息」，他強調，究竟他們是誰？有多關鍵？多可靠？多靈通？講話內容的根據什麼？有什麼不能讓人

KeyboardInterrupt: 

In [ ]:
#將json檔每則新聞作者取出用方法清理
for i in range(0, len(json_data)):  
    if json_data[i]["author"] == []:
        author = []
        print(author)
    else:
        tvbs_author_etl(json_data[i]["author"][0])

In [ ]:
#Streaming從每單一json檔取出新聞作者用方法清理
if j["author"] == []:
    author = []
    print(author)
else:
    tvbs_author_etl(j["author"][0])

In [27]:
# coding=utf-8
import threading, queue, time, urllib
from  urllib import request
baseUrl = 'http://www.pythontab.com/html/pythonjichu/'
urlQueue = queue.Queue()
for i in range(2, 10):
    url = baseUrl + str(i) + '.html'
    urlQueue.put(url)
    #print(url)
def fetchUrl(urlQueue):
    while True:
        try:
            #不阻塞的读取队列数据
            url = urlQueue.get_nowait()
            i = urlQueue.qsize()
        except Exception as e:
            break
        print ('Current Thread Name %s, Url: %s ' % (threading.currentThread().name, url))
        try:
            response = urllib.request.urlopen(url)
            responseCode = response.getcode()
        except Exception as e:
            continue
        if responseCode == 200:
            #抓取内容的数据处理可以放到这里
            #为了突出效果， 设置延时
            time.sleep(1)
if __name__ == '__main__':
    startTime = time.time()
    threads = []
    # 可以调节线程数， 进而控制抓取速度
    threadNum = 2
    for i in range(0, threadNum):
        t = threading.Thread(target=fetchUrl, args=(urlQueue,))
        threads.append(t)
    for t in threads:
        t.start()
    for t in threads:
        #多线程多join的情况下，依次执行各线程的join方法, 这样可以确保主线程最后退出， 且各个线程间没有阻塞
        t.join()
    endTime = time.time()
    print ('Done, Time cost: %s ' %  (endTime - startTime))

Current Thread Name Thread-9, Url: http://www.pythontab.com/html/pythonjichu/2.html 
Current Thread Name Thread-10, Url: http://www.pythontab.com/html/pythonjichu/3.html 
Current Thread Name Thread-10, Url: http://www.pythontab.com/html/pythonjichu/4.html 
Current Thread Name Thread-9, Url: http://www.pythontab.com/html/pythonjichu/5.html 
Current Thread Name Thread-10, Url: http://www.pythontab.com/html/pythonjichu/6.html 
Current Thread Name Thread-9, Url: http://www.pythontab.com/html/pythonjichu/7.html 
Current Thread Name Thread-9, Url: http://www.pythontab.com/html/pythonjichu/8.html 
Current Thread Name Thread-10, Url: http://www.pythontab.com/html/pythonjichu/9.html 
Done, Time cost: 11.269868850708008 
